# Explore OpenFace

This notebook will explore how [OpenFace](https://github.com/TadasBaltrusaitis/OpenFace) can be used to extract action unit features from faces in video files. To run the notebook, **Docker** needs to be installed on the computer and added to the system path. Furthermore, the libraries in the `requirements.txt` file need to be installed. 

Several stock videos will be downloaded as test cases to investigate the feature extraction for **one**, **two**, and **multiple** (>2) faces displayed in a video. The videos are stored in the `/video` directory.

In [1]:
""" Explore OpenFace """

from urllib.request import urlopen, Request
import subprocess
import os
import IPython
import docker
import pandas as pd

client = docker.from_env()

In [2]:
def check_dir_exists(filename):
    """ Create file directory if it does not exist """
    dir = "".join(filename.split("/")[:-1])
    if not os.path.exists(dir):
        os.system(f"mkdir {dir}")
        print(f"Created directory /{dir}")

In [3]:
def download_file_url(url, filename):
    """ Download file from url if it does not exist """
    if not os.path.exists(filename):
        check_dir_exists(filename)
        req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
        file = urlopen(req)
        
        print(f"Downloaded file from {url}")

        with open(filename, "wb") as f:
            f.write(file.read())

        print(f"Saved file to {filename}")
    else:
        print(f"File found at {filename}")

## Using OpenFace with Docker

To run OpenFace in docker, we pull the docker image and run the container as an interactive session that is detached. This allows copying files into the container while running. Then, the feature extraction is performed inside the container and results are copied out of the container before it is closed and removed. 

### Single Face in Video

In [4]:
def extract_features_video_docker(client, filename, logfilename):
    """ 
    Extracts openface features from a single face in a video file using a docker container. 
    Copies video file into the openface docker container.
    Extracts openface features in docker container.
    Saves openface log to a logfile.
    Copies results (processed files) back to working directory. 
    """
    container = client.containers.run("algebr/openface:latest", name="openface", stdin_open=True, tty=True, remove=True, detach=True)
    print("Started docker container")
    
    with open(logfilename, "wb") as logfile:
        subprocess.run(f"docker cp {filename} openface:/home/openface-build", shell=True)
        print("Copied file to docker container")
        _, output = container.exec_run(f"build/bin/FeatureExtraction -f {filename.split('/')[-1]}")
        print("Extracted openface features")
        subprocess.run("docker cp openface:/home/openface-build/processed .", shell=True)
        print("Copied results from docker container to /processed")
        logfile.write(output)
        
    container.stop()
    print("Stopped docker container")

In [5]:
download_file_url("https://www.pexels.com/video/6173219/download/?search_query=&tracking_id=fnzpvugz2cj", "video/test_video_single.mp4")

File found at video/test_video_single.mp4


In [6]:
extract_features_video_docker(client, "video/test_video_single.mp4", "test_video_single.log")

Started docker container
Copied file to docker container
Extracted openface features
Copied results from docker container to /processed
Stopped docker container


The results from OpenFace are stored in the `/processed` directory. It contains the video file `test_video_single.avi` which is overlayed with the extracted facial features (not only action units). The face tracking and feature extraction seems to perform well for a single face which is directed to a point next to the camera. 

In [20]:
IPython.display.Video("processed/test_video_single.mp4")

The directory also contains the extracted features in the file `test_video_single.csv`:

In [8]:
features_single_df = pd.read_csv("processed/test_video_single.csv")

In [9]:
print(features_single_df)

     frame   face_id   timestamp   confidence   success   gaze_0_x   gaze_0_y  \
0        1         0       0.000         0.98         1  -0.531649   0.133721   
1        2         0       0.033         0.98         1  -0.542727   0.216815   
2        3         0       0.067         0.98         1  -0.525011   0.207253   
3        4         0       0.100         0.98         1  -0.527497   0.217476   
4        5         0       0.133         0.98         1  -0.517876   0.212282   
..     ...       ...         ...          ...       ...        ...        ...   
588    589         0      19.600         0.98         1  -0.431405   0.223384   
589    590         0      19.633         0.98         1  -0.425573   0.200113   
590    591         0      19.667         0.98         1  -0.430954   0.210013   
591    592         0      19.700         0.98         1  -0.430658   0.195615   
592    593         0      19.733         0.98         1  -0.420955   0.197427   

      gaze_0_z   gaze_1_x  

The data frame contains many columns for the extracted facial features including the action units (`AUXX_x`). The columns with `_c` appended to the name indicate whether the action unit is present (0/1) and the ones with `_r` indicate the intensity of the unit if present (scale 1 to 5). Because there is only one face present in the video, the `face_id` column only contains a single index (0).

### Two Faces in Video

In [10]:
def extract_features_video_multi_docker(client, filename, logfilename):
    """ 
    Extracts openface features from multiple faces in a video file using a docker container. 
    Copies video file into the openface docker container.
    Extracts openface features in docker container.
    Saves openface log to a logfile.
    Copies results (processed files) back to working directory. 
    """
    container = client.containers.run("algebr/openface:latest", name="openface", stdin_open=True, tty=True, remove=True, detach=True)
    print("Started docker container")
    
    with open(logfilename, "wb") as logfile:
        subprocess.run(f"docker cp {filename} openface:/home/openface-build", shell=True)
        print("Copied file to docker container")
        _, output = container.exec_run(f"build/bin/FaceLandmarkVidMulti -f {filename.split('/')[-1]}")
        print("Extracted openface features")
        subprocess.run("docker cp openface:/home/openface-build/processed .", shell=True)
        print("Copied results from docker container to /processed")
        logfile.write(output)
        
    container.stop()
    print("Stopped docker container")

In [11]:
download_file_url("https://www.pexels.com/video/8847983/download/?search_query=political%20debate&tracking_id=fnzpvugz2cj", "video/test_video_two.mp4")

File found at video/test_video_two.mp4


In [12]:
extract_features_video_multi_docker(client, "video/test_video_two.mp4", "test_video_two.log")

Started docker container
Copied file to docker container
Extracted openface features
Copied results from docker container to /processed
Stopped docker container


The face tracker initially struggels to track the face of the woman on the left side in the video. It does not track the face of the man on the right for the entire video. This could be because the faces are directed away from the camera towards each other (the man's face is less visible than the woman's). This behavior might problematic because in talk shows or debates cameras might view faces from very different angles, althought often alternative camera views might be available.

In [13]:
features_two_df = pd.read_csv("processed/test_video_two.csv")

In [14]:
print(features_two_df)

     frame   face_id   timestamp   confidence   success   gaze_0_x   gaze_0_y  \
0        1         0        0.00         0.62         0   0.000000   0.000000   
1        1         1        0.00         0.03         0   0.000000   0.000000   
2        2         0        0.04         0.88         1   0.252523   0.176967   
3        2         1        0.04         0.03         0   0.000000   0.000000   
4        3         0        0.08         0.03         0   0.000000   0.000000   
..     ...       ...         ...          ...       ...        ...        ...   
634    544         0       21.72         0.98         1   0.590757   0.366819   
635    545         0       21.76         0.98         1   0.585318   0.354208   
636    545         1       21.76         0.03         0   0.000000   0.000000   
637    546         0       21.80         0.98         1   0.578965   0.350431   
638    546         1       21.80         0.03         0   0.000000   0.000000   

      gaze_0_z   gaze_1_x  

This time, there are two unique values in the `face_id` column. However, for the second face (`face_id = 1`), the column `success` indicates that it could not be successfully tracked.

### Multiple Faces in Video

In [15]:
download_file_url("https://www.pexels.com/video/8847713/download/?search_query=political%20debate&tracking_id=fnzpvugz2cj", "video/test_video_multi.mp4")

File found at video/test_video_multi.mp4


In [16]:
extract_features_video_multi_docker(client, "video/test_video_multi.mp4", "test_video_multi.log")

Started docker container
Copied file to docker container
Extracted openface features
Copied results from docker container to /processed
Stopped docker container


The faces of all four persons in the video were succesfully tracked (when not covered). All faces are directed towards the camera.

In [17]:
features_multi_df = pd.read_csv("processed/test_video_multi.csv")

In [18]:
print(features_multi_df)

      frame   face_id   timestamp   confidence   success   gaze_0_x  \
0         1         0        0.00         0.98         1   0.027142   
1         1         1        0.00         0.88         1   0.022368   
2         1         2        0.00         0.98         1   0.353438   
3         1         3        0.00         0.03         0   0.000000   
4         2         0        0.04         0.98         1   0.025669   
...     ...       ...         ...          ...       ...        ...   
2069    536         3       21.40         0.88         1   0.076024   
2070    537         0       21.44         0.98         1   0.087262   
2071    537         1       21.44         0.98         1  -0.349506   
2072    537         2       21.44         0.98         1   0.320851   
2073    537         3       21.44         0.88         1   0.074254   

       gaze_0_y   gaze_0_z   gaze_1_x   gaze_1_y  ...   AU12_c   AU14_c  \
0      0.057760  -0.997961  -0.111140   0.053308  ...      0.0      1.0 

Now, there are four different `face_id` values that seem to be consistent throughout the video. However, the OpenFace [wiki](https://github.com/TadasBaltrusaitis/OpenFace/wiki/Output-Format) remarks that there is now guarantee for consistency (in particular for longer videos).